<a href="https://colab.research.google.com/github/NinhDT22022522/NLP-ASM2/blob/main/SBOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
with open("/content/drive/MyDrive/Colab_Notebooks/NLP/Week2/word2vec/truyen_kieu_data.txt", "r", encoding="utf-8") as f:
    lines = f.readlines()


In [4]:
lines[0]

'1..Trăm năm trong cõi người ta,\n'

In [5]:
import string as pystring

# Định nghĩa các ký tự cần loại bỏ
PUNCT_TO_REMOVE = pystring.punctuation + pystring.digits + "\n"

def clean_text(text):
    """Hàm tùy chỉnh để loại bỏ dấu câu và số"""
    text = text.lower()  # chuyển thành chữ thường
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    return text

# Ví dụ kiểm tra hàm clean_text
print(clean_text(lines[0]))


trăm năm trong cõi người ta


In [12]:
cleaned_lines = [clean_text(line) for line in lines]

In [11]:
import string as pystring
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
import numpy as np
from keras.models import Sequential
from keras.layers import Embedding, Dense, Lambda
import keras.backend as K

In [10]:
def get_centers_and_contexts(corpus, max_window_size=2):
    centers, contexts = [], []
    for line in corpus:
        line = line.split()
        if len(line) <= 2 * max_window_size:
            continue
        for i in range(max_window_size, len(line) - max_window_size):
            centers.append(line[i])
            idxs = list(range(i - max_window_size, i + max_window_size + 1))
            idxs.remove(i)
            contexts.append(" ".join([line[idx] for idx in idxs]))
    return centers, contexts


In [13]:
centers, contexts = get_centers_and_contexts(cleaned_lines)

In [14]:
max_length = 4
embedding_size = 200

tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(cleaned_lines)

vocab_size = len(tokenizer.index_word) + 1

train_seq = tokenizer.texts_to_sequences(contexts)
train_seq_pad = pad_sequences(train_seq, maxlen=max_length, truncating='post', padding='post')

train_labels = [to_categorical(tokenizer.word_index[label], vocab_size) for label in centers]
train_labels = np.array(train_labels)

In [15]:
cbow = Sequential()
cbow.add(Embedding(input_dim=vocab_size, output_dim=embedding_size, input_length=max_length))
cbow.add(Lambda(lambda x: K.mean(x, axis=1), output_shape=(embedding_size,)))
cbow.add(Dense(vocab_size, activation='softmax'))

cbow.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])
cbow.summary()

cbow.fit(train_seq_pad, train_labels, epochs=30, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 4, 200)            479000    
                                                                 
 lambda (Lambda)             (None, 200)               0         
                                                                 
 dense (Dense)               (None, 2395)              481395    
                                                                 
Total params: 960395 (3.66 MB)
Trainable params: 960395 (3.66 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
Epoch 1/30
306/306 [==============================] - 9s 22ms/step - loss: 7.4525 - acc: 0.0183
Epoch 2/30
306/306 [==============================] - 7s 22ms/step - loss: 6.7268 - acc: 0.0209
Epoch 3/30
306/306 [==============================] - 4s 14ms/step - loss: 6.5661 - acc: 0.0

In [16]:
sample_text = 'trăm năm cõi người'
sample_seq = tokenizer.texts_to_sequences([sample_text])
sample_seq_pad = pad_sequences(sample_seq, maxlen=max_length, truncating='post', padding='post')
predicted_word_idx = np.argmax(cbow.predict(sample_seq_pad))
predicted_word = tokenizer.index_word[predicted_word_idx]

print(f"Predicted center word: {predicted_word}")

1/1 [==============================] - 0s 179ms/step
Predicted center word: trong
